In [2]:
from pyspark.sql import SparkSession
import mysql.connector

In [3]:
# Spark 세션 시작
spark = SparkSession.builder \
    .appName("CSV to MariaDB") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/23 08:27:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/23 08:27:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/23 08:27:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
connection = mysql.connector.connect(
    host="13.124.90.34",
    user="root",
    password="enqnrhkwk108",
    database="S11P21D108",
    charset="utf8mb4",  # 문자셋 설정
    collation="utf8mb4_general_ci",  # collation 설정
    autocommit=False,  # Auto-commit을 끄고 트랜잭션 처리
    connection_timeout=28800,  # 타임아웃 증가
)

In [5]:
# CSV 파일 읽기
area_df = spark.read.option("encoding", "cp949").csv("hdfs://master:9000/input/서울시_상권분석서비스(추정매출-상권).csv", header=True, inferSchema=True)


In [6]:
# CSV 파일 읽기
dong_df = spark.read.option("encoding", "cp949").csv("hdfs://master:9000/input/서울시_상권분석서비스(추정매출-행정동).csv", header=True, inferSchema=True)


In [7]:
# CSV 파일 읽기
gu_df = spark.read.option("encoding", "cp949").csv("hdfs://master:9000/input/서울시_상권분석서비스(추정매출-자치구).csv", header=True, inferSchema=True)


In [8]:
# '서비스_업종_코드'와 '서비스_업종_코드_명'의 중복 제거
unique_service_codes = area_df.select('서비스_업종_코드', '서비스_업종_코드_명').distinct()

# 결과 출력
unique_service_codes.show()

+----------------+--------------------+
|서비스_업종_코드| 서비스_업종_코드_명|
+----------------+--------------------+
|        CS100009|       호프-간이주점|
|        CS300024|       운동/경기용품|
|        CS300029|            애완동물|
|        CS200030|          피부관리실|
|        CS300001|            슈퍼마켓|
|        CS300020|                서적|
|        CS300007|            육류판매|
|        CS100007|          치킨전문점|
|        CS200016|              당구장|
|        CS200019|                PC방|
|        CS300031|                가구|
|        CS200008|              한의원|
|        CS300026|                완구|
|        CS200003|            예술학원|
|        CS200025|          자동차수리|
|        CS300019|            의료기기|
|        CS300003|컴퓨터및주변장치판매|
|        CS200005|         스포츠 강습|
|        CS200024|          스포츠클럽|
|        CS100008|          분식전문점|
+----------------+--------------------+
only showing top 20 rows



In [9]:
# '서비스_업종_코드'와 '서비스_업종_코드_명'의 중복 제거
unique_service_codes = dong_df.select('서비스_업종_코드', '서비스_업종_코드_명').distinct()

# 결과 출력
unique_service_codes.show()

+----------------+--------------------+
|서비스_업종_코드| 서비스_업종_코드_명|
+----------------+--------------------+
|        CS100009|       호프-간이주점|
|        CS300024|       운동/경기용품|
|        CS300029|            애완동물|
|        CS200030|          피부관리실|
|        CS300001|            슈퍼마켓|
|        CS300020|                서적|
|        CS300007|            육류판매|
|        CS100007|          치킨전문점|
|        CS200016|              당구장|
|        CS200019|                PC방|
|        CS200008|              한의원|
|        CS300031|                가구|
|        CS300026|                완구|
|        CS200003|            예술학원|
|        CS200025|          자동차수리|
|        CS300019|            의료기기|
|        CS300003|컴퓨터및주변장치판매|
|        CS200005|         스포츠 강습|
|        CS200024|          스포츠클럽|
|        CS100008|          분식전문점|
+----------------+--------------------+
only showing top 20 rows



In [10]:
# '서비스_업종_코드'와 '서비스_업종_코드_명'의 중복 제거
unique_service_codes = gu_df.select('서비스_업종_코드', '서비스_업종_코드_명').distinct()

# 결과 출력
unique_service_codes.show()

+----------------+--------------------+
|서비스_업종_코드| 서비스_업종_코드_명|
+----------------+--------------------+
|        CS100009|       호프-간이주점|
|        CS300024|       운동/경기용품|
|        CS300029|            애완동물|
|        CS200030|          피부관리실|
|        CS300001|            슈퍼마켓|
|        CS300020|                서적|
|        CS300007|            육류판매|
|        CS100007|          치킨전문점|
|        CS200016|              당구장|
|        CS200019|                PC방|
|        CS300031|                가구|
|        CS200008|              한의원|
|        CS300026|                완구|
|        CS200003|            예술학원|
|        CS200025|          자동차수리|
|        CS300019|            의료기기|
|        CS300003|컴퓨터및주변장치판매|
|        CS200005|         스포츠 강습|
|        CS200024|          스포츠클럽|
|        CS100008|          분식전문점|
+----------------+--------------------+
only showing top 20 rows



In [11]:
def sh_dong(row):
          # 삽입할 SQL 쿼리 정의
            sql = """
            INSERT INTO service (service_code,service_name )
            VALUES (%s, %s)
            """
            
            # 데이터 삽입
            cursor.execute(sql, (
                row['서비스_업종_코드'],  # service_code
              row['서비스_업종_코드_명']
            ))

In [12]:
cursor = connection.cursor()

# DataFrame의 각 행을 MariaDB에 삽입
for row in unique_service_codes.collect():
    sh_dong(row)

# 변경사항 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()